In [1]:
!pip install --upgrade numexpr

In [2]:
!pip install --upgrade bottleneck

In [68]:
import pandas as pd
import numpy as np
from itertools import combinations
from itertools import product

In [69]:
data = {
    "N_candidat": ["0001", "0002", "0003", "0004", "0005"],
    "S1":[15, 13, 16, 17, 13],
    "S2": [11, 9, 8, 12, 10],
    "S3":[14, 12, 11, 15, 13],
    "S4": [16, 15, 15, 18, 15],
    "S5": [11, 11, 11, 11, 11],

    "Logique": [1, 1, 1, 0, 0],
    "BDD": [1, 1, 1, 0, 0],
    "PS": [1, 1, 1, 1, 0],
    "Moyenne": [2, 0, 1, 2, 0],
    "Eligible": [True, True, True, False, False],
    "Decision": ['A','R','L','R','R']
}

df= pd.DataFrame(data)
df

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,11,14,16,11,1,1,1,2,True,A
1,0002,13,9,12,15,11,1,1,1,0,True,R
2,0003,16,8,11,15,11,1,1,1,1,True,L
3,0004,17,12,15,18,11,0,0,1,2,False,R
4,0005,13,10,13,15,11,0,0,0,0,False,R


In [70]:

def AC1(N_candidat, X, decision, df=df):
  candidat=df[df.N_candidat==N_candidat].copy()
  X_var=list(X.keys())

  # Retourner False dans le cas où le candidat n'existe pas dans la base de données utilisée
  if (len(candidat)!=1):
    return False

  # Retourner False dans le cas où la décision donnée (phi) ne correspond pas à la décision prise à l'égard du candidat
  if (candidat.Decision.iloc[0]!=decision):
    return False

  # Retourner False dans le cas où une des valeurs (notes) du candidat ne correspond pas à celle du vecteur x (tel que X=x)
  for var in X_var :
    if str(candidat.loc[:,[var]].iloc[0,0]) != str(X[var]):
      return False

  return True

print(" Cas succès 1 : ", AC1("0001", {"Logique":"1"}, 'A'))
print(" Cas succès 2 : ", AC1("0001", {"S1":"15", "BDD":"1", "Moyenne":"2", 'Eligible': True}, 'A'))
print(" Cas succès 3 : ", AC1("0004", {"S2":"12", "BDD":"0"}, 'R'))

print(" Cas échec 1 : ", AC1("0001", {"S1":"15", "BDD":"1", "Moyenne":"2", 'Eligible': False}, 'A'))
print(" Cas échec 2 : ", AC1("xxxx", {"S1":"15", "BDD":"1", "Moyenne":"2", 'Eligible': True}, 'A'))



 Cas succès 1 :  True
 Cas succès 2 :  True
 Cas succès 3 :  True
 Cas échec 1 :  False
 Cas échec 2 :  False


In [72]:
coeffs= [1, 1, 1, 1, 1] #calcul de la Moyenne
seuils = [12, 12, 12] # Calcul du status d'éligibilité (logique, BDD, PS)
thresholds = [12, 14] # prendre la décision

def med(S1, S2, S3, S4, S5, Logique, BDD, PS, coeffs=coeffs, seuils=seuils, thresholds2=thresholds):
    moyenne = (S1 * coeffs[0] + S2 * coeffs[1] + S3 * coeffs[2] + S4 * coeffs[3] + S5 * coeffs[4]) / sum(coeffs)
    e = (Logique*BDD*PS==1)
    m = (moyenne>thresholds2[1])*2 + (thresholds2[0] <= moyenne <= thresholds2[1])
    if e and m==2:
        d= 'A'
    elif e and m==1:
        d='L'
    else:
        d='R'
    return m, e, d


In [131]:
def AC2(N_candidat, X, decision, df=df):
    candidat=df[df.N_candidat==N_candidat].copy()
    if (len(candidat)==0):
        return False
    vals=candidat.iloc[0].to_dict()
    X_var=list(X.keys())
    
    #W=list(set(candidat.columns)-set(X_var)) #optionnel

    variable_domains = {
        "S1": range(0, 21),
        "S2": range(0, 21),
        "S3": range(0, 21),
        "S4": range(0, 21),
        "S5": range(0, 21),
        "BDD": [0, 1],
        "Logique": [0, 1],
        "PS": [0, 1],
        "Moyenne":[0,1,2],
        "Eligible":[True,False]
    }
    def generate_combinations(X, variable_domains):
        # Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
        combinations = product(*[variable_domains[var] for var in X])

        # Exclure la combinaison identique à X
        return [combination for combination in combinations if combination != tuple(X.values())]

    combinations = generate_combinations(X, variable_domains)
    
    # Vérifier chaque combinaison
    for combination in combinations:
        x_prime = dict(zip(X_var, combination))
        for k in x_prime.keys():
            vals[k]=x_prime[k]
        m,e,d=med(**{k: v for k, v in vals.items() if k in ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']})

        if ('Eligible' in x_prime and x_prime['Eligible']!=e) or ('Moyenne' in x_prime and x_prime['Moyenne']!=m) :
            continue
        if d!=decision:
            print(f"AC2 est vérifié pour une affectation x_prime: {x_prime}")
            return True
    return False

In [134]:
print(" Cas succès 1 : ", AC2("0001", {"Logique":1}, 'A'))
print(" Cas succès 2 : ", AC2("0004", {"Logique":0, "BDD":0}, 'R'))
print(" Cas succès 3 : ", AC2("0002", {"S2":9,'S5':11}, 'R'))
print(" Cas succès 4 : ", AC2("0005", {"Logique":0,"BDD":0,"PS":0, "Moyenne":0}, 'R'))

print(" Cas échec 1 : ", AC2("0005", {"BDD":0,"PS":0, "Moyenne":0}, 'R'))
print(" Cas échec 2 : ", AC2("xxxx", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A'))

AC2 est vérifié pour une affectation x_prime: {'Logique': 0}
 Cas succès 1 :  True
AC2 est vérifié pour une affectation x_prime: {'Logique': 1, 'BDD': 1}
 Cas succès 2 :  True
AC2 est vérifié pour une affectation x_prime: {'S2': 0, 'S5': 20}
 Cas succès 3 :  True
AC2 est vérifié pour une affectation x_prime: {'Logique': 1, 'BDD': 1, 'PS': 1, 'Moyenne': 1}
 Cas succès 4 :  True
 Cas échec 1 :  False
 Cas échec 2 :  False


In [86]:
def BC3(X1, X2, df=None):
    
    # Check si les deux X1 et X2 contiennent les memes variables
    if set(X1.keys()) != set(X2.keys()):
        return False
    
    # Check que l'intersection des deux affectations est vide
    for key in X1.keys():
        if X1[key] == X2[key]:
            return False
    
    return True

In [87]:
print(" Cas succès 1 : ", BC3({"S2":12, "BDD":0},{"S2":11, "BDD":1}))

print(" Cas échec 1 : ", BC3({"S1":15, "BDD":1, "Moyenne":2, 'Eligible': False},{"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))
print(" Cas échec 2 : ", BC3({"S1":15, "BDD":1, 'Eligible': False},{"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))

 Cas succès 1 :  True
 Cas échec 1 :  False
 Cas échec 2 :  False


In [6]:
def BCi1_3(N_candidat=None, X=None):
  return True

In [7]:
def BC4(N_candidat, X, df=df):

  # S'assurer que X verifie tous les Bci1-3
  if BCi1_3(X):
    return False

  candidat=df[df.N_candidat==N_candidat].copy()
  X_var=list(X.keys())
  V_var=list(candidat.columns)[1:-1] # Prendre toutes les variables qui décrivent un candidat sauf N_candidat et Décision
  V_minus_X_var=[var for var in V_var if not var in X_var]

  # Trouver toutes les combinaisons possibles de longueur 1 à la longueur de la liste
  all_combinaisons_var = []
  print(V_minus_X_var)
  for nb_var in range(1, len(V_minus_X_var) + 1):
      all_combinaisons_var.extend(all_combinaisons_var(V_minus_X_var, nb_var))

  for var in all_combinaisons_var:
    if (BCi1_3(N_candidat, X+list(var))):
      return False # Retourner False car on aurait trouvé un superset qui verifie tous les BC1-3

  return True



In [8]:
# ma_liste = [1, 2, 3]
# combinaisons = []
# for taille in range(1, len(ma_liste) + 1):
#     combinaisons.extend(combinations(ma_liste, taille))
# combinaisons